In [2]:
########## Import Libararies
import os
import pickle
import glob
from tqdm.notebook import tqdm

import numpy as np
from skimage import measure
import scipy.ndimage
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline

import open3d as o3d
from compas.geometry import trimesh_remesh
from compas.datastructures import Mesh
import nibabel as nib

from util import *

import csv
import copy

flag = "manual" # manual or synthseg

import csv
def read_sub(file_path):
    data = []
    with open(file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        for index, row in enumerate(csv_reader):
            if index==0: continue
            else:
                id= row[0]
                data.append(id)
    return data
common_sub_csv =  rf"D:\NextCloud\원정\2024\Data-to-KAIST\sub.csv"
common_dict = read_sub(common_sub_csv)
print(len(common_dict), common_dict)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
693 ['LBC360002', 'LBC360003', 'LBC360004', 'LBC360006', 'LBC360007', 'LBC360008', 'LBC360009', 'LBC360013', 'LBC360015', 'LBC360017', 'LBC360020', 'LBC360021', 'LBC360022', 'LBC360023', 'LBC360025', 'LBC360026', 'LBC360029', 'LBC360030', 'LBC360031', 'LBC360032', 'LBC360034', 'LBC360036', 'LBC360038', 'LBC360040', 'LBC360041', 'LBC360042', 'LBC360043', 'LBC360045', 'LBC360048', 'LBC360049', 'LBC360051', 'LBC360052', 'LBC360055', 'LBC360056', 'LBC360058', 'LBC360059', 'LBC360061', 'LBC360063', 'LBC360066', 'LBC360067', 'LBC360069', 'LBC360070', 'LBC360072', 'LBC360074', 'LBC360076', 'LBC360079', 'LBC360081', 'LBC360082', 'LBC360083', 'LBC360085', 'LBC360086', 'LBC360087', 'LBC360089', 'LBC360091', 'LBC360092', 'LBC360097', 'LBC360100', 'LBC360102', 'LBC360104', 'LBC360106', 'LBC360107', 'LBC360108', 'LBC360109', 'LBC360

In [ ]:
######### Read Hippo Temp Mesh #########
classes = [10,11,12,13,17,18,26,28,49,50,51,52,53,54,58,60]
temp_meshes = []
temp_pcds= []
for c in classes:
  temp_mesh = o3d.io.read_triangle_mesh(rf"D:\IdeaProjects\IITP_synth\lv-parametric-modelling\whole_brain_structure\temp_meshes\{c}_temp.obj")
  temp_meshes.append(temp_mesh)
  temp_pcd = o3d.geometry.PointCloud(points=temp_mesh.vertices)
  temp_pcd.normals = temp_mesh.vertex_normals
  temp_pcds.append(temp_pcd)


scan_path = rf"D:\Data-to-KAIST\masks_npy\synthseg_wave1"
filelist = glob.glob(rf"{scan_path}\*.npy")
subs = os.listdir(scan_path)
save_path = r"D:\Data-to-KAIST\Task4) longi-subcortical"

for index, file in tqdm(enumerate(filelist)):

	masks = np.load(file)	
	sub= file.split("\\")[-1].split(".")[0]		
	for c_ind, c in enumerate(classes):
		target_mask = (masks == c).astype(int)
		target_pcd = voxel_to_bd_pcd(target_mask)
		target_pcd = pcd_to_orgin(target_pcd)

		regist_mesh_pcd(target_pcd, temp_pcds[c_ind],normal=False)
		target_pcd.paint_uniform_color([1, 0.706, 0])
		temp_pcds[c_ind].paint_uniform_color([0, 0.651, 0.929])
		# o3d.visualization.draw_geometries([temp_pcds[c_ind], target_pcd])

		data = {}
		data['vertices'] = np.asarray(temp_meshes[c_ind].vertices)
		data['faces'] = np.asarray(temp_meshes[c_ind].triangles)
		data['target'] = np.asarray(target_pcd.points)
		with open(rf'{save_path}\{c}\{sub}_{c}_scan1_opt_data.pkl', 'wb') as f:
			pickle.dump(data, f)



0it [00:00, ?it/s]

In [ ]:
import os
classes = [10,11,12,13,17,18,26,28,49,50,51,52,53,54,58,60]
for c in classes:
	os.makedirs(rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan4_train_data\{c}",exist_ok=True)

In [ ]:
vert = np.array(hippo_l_mesh.triangles)
print(vert.shape)

(6170, 3)


In [ ]:
import numpy as np
classes = [10,11,12,13,17,18,26,28,49,50,51,52,53,54,58,60]
for label in classes:
	optim_path = rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan1_optimed_data\{label}_scan1\out"
	train_path = rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan1_train_data\{label}"
	subs = os.listdir(optim_path)
	for ind, sub in enumerate(subs):
		with open(rf'D:\Data-to-KAIST\Task4) longi-subcortical\scan1_train_data\{label}\{sub}_{label}_scan1_opt_data.pkl', 'rb') as f:
			data = pickle.load(f)
		vert = data['vertices']
		face = data['faces']
		target = data['target']
		face0 = face.copy()
		face[:,2],face[:,0] = face0[:,0],face0[:,2]
	
		target_p = o3d.geometry.PointCloud(points = o3d.utility.Vector3dVector(target))

		deformed_mesh = o3d.geometry.TriangleMesh(vertices = o3d.utility.Vector3dVector(vert), triangles = o3d.utility.Vector3iVector(face))
		newvert= np.load(glob.glob(rf"{optim_path}\{sub}\5000*.npy")[0])[0]
		deformed_mesh.vertices = o3d.utility.Vector3dVector(newvert)
		print(vert.shape)
		o3d.visualization.draw_geometries([deformed_mesh, target_p])
		if ind >3: 
			print(label)
			break


(1456, 3)
(1456, 3)
(1456, 3)
(1456, 3)
(1456, 3)
10
(1423, 3)
(1423, 3)
(1423, 3)
(1423, 3)
(1423, 3)
11
(1549, 3)
(1549, 3)
(1549, 3)
(1549, 3)
(1549, 3)
12
(613, 3)
(613, 3)
(613, 3)
(613, 3)
(613, 3)
13
(1490, 3)
(1490, 3)
(1490, 3)
(1490, 3)
(1490, 3)
17
(623, 3)
(623, 3)
(623, 3)
(623, 3)
(623, 3)
18
(336, 3)
(336, 3)
(336, 3)
(336, 3)
(336, 3)
26
(1307, 3)
(1307, 3)
(1307, 3)
(1307, 3)
(1307, 3)
28
(1412, 3)
(1412, 3)
(1412, 3)
(1412, 3)
(1412, 3)
49
(1492, 3)
(1492, 3)
(1492, 3)
(1492, 3)
(1492, 3)
50
(1521, 3)
(1521, 3)
(1521, 3)
(1521, 3)
(1521, 3)
51
(600, 3)
(600, 3)
(600, 3)
(600, 3)
(600, 3)
52
(1492, 3)
(1492, 3)
(1492, 3)
(1492, 3)
(1492, 3)
53
(631, 3)
(631, 3)
(631, 3)
(631, 3)
(631, 3)
54
(325, 3)
(325, 3)
(325, 3)
(325, 3)
(325, 3)
58
(1303, 3)
(1303, 3)
(1303, 3)
(1303, 3)
(1303, 3)
60


# Scan1 > 2


In [ ]:

######### Read Hippo Temp Mesh #########
classes = [58]
for label in classes:
	temp_mesh = o3d.io.read_triangle_mesh(rf"D:\IdeaProjects\IITP_synth\lv-parametric-modelling\whole_brain_structure\temp_meshes\{label}_temp.obj")
	temp_mesh.compute_vertex_normals()
	temp_mesh.paint_uniform_color([0.7, 1.0, 0.7])
	# o3d.visualization.draw_geometries([temp_mesh])

	temp_pcd = o3d.geometry.PointCloud(points=temp_mesh.vertices)
	temp_pcd.normals = temp_mesh.vertex_normals

	# o3d.visualization.draw_geometries([temp_mesh, R_temp_mesh])

	# filelist = glob.glob("W:\LBC1936_SynthSeg_masks\Scan1Wave2_T1W\*\*SynthSeg.nii.gz") #synthseg 

	save_path = rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan2_train_data\{label}"
	subs = os.listdir(save_path)

	optim_path = r"D:\Data-to-KAIST\masks_npy\synthseg_wave2"

	past_optim_path_l = rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan1_optimed_data\{label}_scan1\out"


	subs= os.listdir(optim_path)
		
	for index, sub in tqdm(enumerate(subs)):
		masks = np.load(rf"{optim_path}\{sub}")	
		target_mask = (masks == label).astype(int)
		target_pcd = voxel_to_bd_pcd(target_mask)
		target_pcd = pcd_to_orgin(target_pcd)
		sub= sub.split(".")[0]
		if os.path.isdir(rf"{past_optim_path_l}\{sub}"):

			past_mesh = copy.deepcopy(temp_mesh)
			past_mesh.vertices = o3d.utility.Vector3dVector(np.load(glob.glob(rf"{past_optim_path_l}\{sub}\5000*.npy")[0])[0])
			past_mesh.compute_vertex_normals()
			past_pcd = o3d.geometry.PointCloud(points = past_mesh.vertices)
			past_pcd.normals = past_mesh.vertex_normals

			stand_pcd= past_pcd
			stand_mesh= past_mesh
		else:
			stand_pcd = temp_pcd
			stand_mesh = temp_mesh


		regist_mesh_pcd(target_pcd, stand_pcd, normal=False)
		target_pcd.paint_uniform_color([1, 0.706, 0])
		temp_pcd.paint_uniform_color([0, 0.651, 0.929])
		# o3d.visualization.draw_geometries([target_pcd, stand_mesh])


		# Save Hippo_L optimization data
		data = {}
		data['vertices'] = np.asarray(stand_mesh.vertices)
		data['faces'] = np.asarray(stand_mesh.triangles)
		data['target'] = np.asarray(target_pcd.points)
		with open(rf'{save_path}\{sub}_{label}_scan2_opt_data.pkl', 'wb') as f:
			pickle.dump(data, f)
		# if index>2: break



0it [00:00, ?it/s]

In [ ]:
o3d.visualization.draw_geometries([hippo_l_mesh, hippo_l_pcd, stand_mesh_l])
o3d.visualization.draw_geometries([hippo_r_mesh, hippo_r_pcd, stand_mesh_r])

# Scan 2 > 3

In [4]:

######### Read Hippo Temp Mesh #########
classes = [10,11,12,13,17,18,26,28,49,50,51,52,53,54,58,60]
classes= [10,11,12,13,17,18,26,28]
for label in classes:
	temp_mesh = o3d.io.read_triangle_mesh(rf"D:\IdeaProjects\IITP_synth\lv-parametric-modelling\whole_brain_structure\temp_meshes\{label}_temp.obj")
	temp_mesh.compute_vertex_normals()
	temp_mesh.paint_uniform_color([0.7, 1.0, 0.7])
	# o3d.visualization.draw_geometries([temp_mesh])

	temp_pcd = o3d.geometry.PointCloud(points=temp_mesh.vertices)
	temp_pcd.normals = temp_mesh.vertex_normals

	# o3d.visualization.draw_geometries([temp_mesh, R_temp_mesh])

	# filelist = glob.glob("W:\LBC1936_SynthSeg_masks\Scan1Wave2_T1W\*\*SynthSeg.nii.gz") #synthseg 

	save_path = rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan3_train_data\{label}"
	subs = os.listdir(save_path)

	optim_path = r"D:\Data-to-KAIST\masks_npy\synthseg_wave3"

	past_optim_path_l = rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan2_optimed_data\{label}_scan2\out"
	past_past_optim_path_l = rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan1_optimed_data\{label}_scan1\out"


	subs= os.listdir(optim_path)
	past_num, past_past_num, temp_num = 0, 0, 0
		
	for index, sub in tqdm(enumerate(subs)):
		masks = np.load(rf"{optim_path}\{sub}")	
		target_mask = (masks == label).astype(int)
		target_pcd = voxel_to_bd_pcd(target_mask)
		target_pcd = pcd_to_orgin(target_pcd)
		sub= sub.split(".")[0]
		if os.path.isdir(rf"{past_optim_path_l}\{sub}"):

			past_mesh = copy.deepcopy(temp_mesh)
			past_mesh.vertices = o3d.utility.Vector3dVector(np.load(glob.glob(rf"{past_optim_path_l}\{sub}\5000*.npy")[0])[0])
			past_mesh.compute_vertex_normals()
			past_pcd = o3d.geometry.PointCloud(points = past_mesh.vertices)
			past_pcd.normals = past_mesh.vertex_normals

			stand_pcd= past_pcd
			stand_mesh= past_mesh
			past_num+=1
		elif os.path.isdir(rf"{past_past_optim_path_l}\{sub}"):

			past_mesh = copy.deepcopy(temp_mesh)
			past_mesh.vertices = o3d.utility.Vector3dVector(np.load(glob.glob(rf"{past_past_optim_path_l}\{sub}\5000*.npy")[0])[0])
			past_mesh.compute_vertex_normals()
			past_pcd = o3d.geometry.PointCloud(points = past_mesh.vertices)
			past_pcd.normals = past_mesh.vertex_normals

			stand_pcd= past_pcd
			stand_mesh= past_mesh
			past_past_num+=1
		else:
			stand_pcd = temp_pcd
			stand_mesh = temp_mesh
			temp_num+=1

		regist_mesh_pcd(target_pcd, stand_pcd, normal=False)
		target_pcd.paint_uniform_color([1, 0.706, 0])
		temp_pcd.paint_uniform_color([0, 0.651, 0.929])
		# o3d.visualization.draw_geometries([target_pcd, stand_mesh])


		# Save Hippo_L optimization data
		# data = {}
		# data['vertices'] = np.asarray(stand_mesh.vertices)
		# data['faces'] = np.asarray(stand_mesh.triangles)
		# data['target'] = np.asarray(target_pcd.points)
		# with open(rf'{save_path}\{sub}_{label}_scan3_opt_data.pkl', 'wb') as f:
		# 	pickle.dump(data, f)
		# if index>2: break
	print(past_num, past_past_num, temp_num)



0it [00:00, ?it/s]

354 26 6


0it [00:00, ?it/s]

KeyboardInterrupt: 

# 3>4

In [9]:

######### Read Hippo Temp Mesh #########
classes = [10,11,12,13,17,18,26,28,49,50,51,52,53,54,58,60]
classes= [10,11,12,13,17,18,26,28,49,50,51,52,53,54,58,60]
for label in classes:
	temp_mesh = o3d.io.read_triangle_mesh(rf"D:\IdeaProjects\IITP_synth\lv-parametric-modelling\whole_brain_structure\temp_meshes\{label}_temp.obj")
	temp_mesh.compute_vertex_normals()
	temp_mesh.paint_uniform_color([0.7, 1.0, 0.7])
	# o3d.visualization.draw_geometries([temp_mesh])

	temp_pcd = o3d.geometry.PointCloud(points=temp_mesh.vertices)
	temp_pcd.normals = temp_mesh.vertex_normals

	# o3d.visualization.draw_geometries([temp_mesh, R_temp_mesh])

	# filelist = glob.glob("W:\LBC1936_SynthSeg_masks\Scan1Wave2_T1W\*\*SynthSeg.nii.gz") #synthseg 

	save_path = rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan4_train_data\{label}"
	subs = os.listdir(save_path)

	optim_path = r"D:\Data-to-KAIST\masks_npy\synthseg_wave4"

	past_optim_path_l = rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan3_optimed_data\{label}_scan3\out"


	subs= os.listdir(optim_path)
	past_num, temp_num = 0,0
	for index, sub in tqdm(enumerate(subs)):
		masks = np.load(rf"{optim_path}\{sub}")	
		target_mask = (masks == label).astype(int)
		target_pcd = voxel_to_bd_pcd(target_mask)
		target_pcd = pcd_to_orgin(target_pcd)
		sub= sub.split(".")[0]
		if os.path.isdir(rf"{past_optim_path_l}\{sub}"):

			past_mesh = copy.deepcopy(temp_mesh)
			past_mesh.vertices = o3d.utility.Vector3dVector(np.load(glob.glob(rf"{past_optim_path_l}\{sub}\5000*.npy")[0])[0])
			past_mesh.compute_vertex_normals()
			past_pcd = o3d.geometry.PointCloud(points = past_mesh.vertices)
			past_pcd.normals = past_mesh.vertex_normals

			stand_pcd= past_pcd
			stand_mesh= past_mesh
			past_num+=1
		else:
			stand_pcd = temp_pcd
			stand_mesh = temp_mesh
			temp_num+=1
   

		regist_mesh_pcd(target_pcd, stand_pcd, normal=False)
		target_pcd.paint_uniform_color([1, 0.706, 0])
		temp_pcd.paint_uniform_color([0, 0.651, 0.929])

		# Save optimization data
		data = {}
		data['vertices'] = np.asarray(stand_mesh.vertices)
		data['faces'] = np.asarray(stand_mesh.triangles)
		data['target'] = np.asarray(target_pcd.points)
		with open(rf'{save_path}\{sub}_{label}_scan4_opt_data.pkl', 'wb') as f:
			pickle.dump(data, f)
   
		# o3d.visualization.draw_geometries([target_pcd, stand_mesh])
		# if index>2: break
	print(past_num, temp_num)



0it [00:00, ?it/s]

278 23


0it [00:00, ?it/s]

278 23


0it [00:00, ?it/s]

...

278 23


0it [00:00, ?it/s]

278 23


0it [00:00, ?it/s]

278 23


# 4>5

In [ ]:
######### Read Hippo Temp Mesh #########
classes = [10,11,12,13,17,18,26,28,49,50,51,52,53,54,58,60]
classes= [49,50,51,52,53,54,58,60]
for label in classes:
	temp_mesh = o3d.io.read_triangle_mesh(rf"D:\IdeaProjects\IITP_synth\lv-parametric-modelling\whole_brain_structure\temp_meshes\{label}_temp.obj")
	temp_mesh.compute_vertex_normals()
	temp_mesh.paint_uniform_color([0.7, 1.0, 0.7])
	# o3d.visualization.draw_geometries([temp_mesh])

	temp_pcd = o3d.geometry.PointCloud(points=temp_mesh.vertices)
	temp_pcd.normals = temp_mesh.vertex_normals

	# o3d.visualization.draw_geometries([temp_mesh, R_temp_mesh])
	# filelist = glob.glob("W:\LBC1936_SynthSeg_masks\Scan1Wave2_T1W\*\*SynthSeg.nii.gz") #synthseg 

	save_path = rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan5_train_data\{label}"
	subs = os.listdir(save_path)

	optim_path = r"D:\Data-to-KAIST\masks_npy\synthseg_wave5"

	past_optim_path_l = rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan4_optimed_data\{label}_scan4\out"
	past_past_optim_path_l = rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan3_optimed_data\{label}_scan3\out"
	past_past_past_optim_path_l = rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan1_optimed_data\{label}_scan1\out"


	subs= os.listdir(optim_path)
	
	past_num,past_past_num,temp_num=0,0,0
	for index, sub in tqdm(enumerate(subs)):
		masks = np.load(rf"{optim_path}\{sub}")	
		target_mask = (masks == label).astype(int)
		target_pcd = voxel_to_bd_pcd(target_mask)
		target_pcd = pcd_to_orgin(target_pcd)
		sub= sub.split(".")[0]
		if os.path.isdir(rf"{past_optim_path_l}\{sub}"):

			past_mesh = copy.deepcopy(temp_mesh)
			past_mesh.vertices = o3d.utility.Vector3dVector(np.load(glob.glob(rf"{past_optim_path_l}\{sub}\5000*.npy")[0])[0])
			past_mesh.compute_vertex_normals()
			past_pcd = o3d.geometry.PointCloud(points = past_mesh.vertices)
			past_pcd.normals = past_mesh.vertex_normals

			stand_pcd= past_pcd
			stand_mesh= past_mesh
			past_num+=1
		elif os.path.isdir(rf"{past_past_optim_path_l}\{sub}"):

			past_mesh = copy.deepcopy(temp_mesh)
			past_mesh.vertices = o3d.utility.Vector3dVector(np.load(glob.glob(rf"{past_past_optim_path_l}\{sub}\5000*.npy")[0])[0])
			past_mesh.compute_vertex_normals()
			past_pcd = o3d.geometry.PointCloud(points = past_mesh.vertices)
			past_pcd.normals = past_mesh.vertex_normals

			stand_pcd= past_pcd
			stand_mesh= past_mesh
			past_past_num+=1
		elif os.path.isdir(rf"{past_past_past_optim_path_l}\{sub}"):

			past_mesh = copy.deepcopy(temp_mesh)
			past_mesh.vertices = o3d.utility.Vector3dVector(np.load(glob.glob(rf"{past_past_past_optim_path_l}\{sub}\5000*.npy")[0])[0])
			past_mesh.compute_vertex_normals()
			past_pcd = o3d.geometry.PointCloud(points = past_mesh.vertices)
			past_pcd.normals = past_mesh.vertex_normals

			stand_pcd= past_pcd
			stand_mesh= past_mesh
			temp_num+=1


		regist_mesh_pcd(target_pcd, stand_pcd, normal=False)
		target_pcd.paint_uniform_color([1, 0.706, 0])
		temp_pcd.paint_uniform_color([0, 0.651, 0.929])


		# Save Hippo_L optimization data
		# data = {}
		# data['vertices'] = np.asarray(stand_mesh.vertices)
		# data['faces'] = np.asarray(stand_mesh.triangles)
		# data['target'] = np.asarray(target_pcd.points)
		# with open(rf'{save_path}\{sub}_{label}_scan3_opt_data.pkl', 'wb') as f:
		# 	pickle.dump(data, f)

		o3d.visualization.draw_geometries([target_pcd, stand_mesh])
		if index>2: break
	print(past_num, past_past_num, temp_num)



0it [00:00, ?it/s]

KeyboardInterrupt: 